<a href="https://colab.research.google.com/github/agersowitz/ADG/blob/master/Copy_of_05_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 5

# Weeks 8 & 9 - Pandas
* In this homework assignment, you will explore and analyze a public dataset of your choosing. Since this assignment is “open-ended” in nature, you are free to expand upon the requirements below. However, you must meet the minimum requirments as indicated in each section. 

* You must use Pandas as the **primary tool** to process your data.

* The preferred method for this analysis is in a .ipynb file. Feel free to use whichever platform of your choosing.  
 * https://www.youtube.com/watch?v=inN8seMm7UI (Getting started with Colab).

* Your data should need some "work", or be considered "dirty".  You must show your skills in data cleaning/wrangling.

### Some data examples:
•	https://www.data.gov/

•	https://opendata.cityofnewyork.us/

•	https://datasetsearch.research.google.com/

•	https://archive.ics.uci.edu/ml/index.php

### Resources:

•	https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html 

•	https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html


### Headings or comments
**You are required to make use of comments, or headings for each section.  You must explain what your code is doing, and the results of running your code.**  Act as if you were giving this assignment to your manager - you must include clear and descriptive information for each section.




# Research Question

In this section, please state the question you are trying to solve.  As mentioned, this assignment is opened-ended, so you are free to choose a research question, and corrosponding data sources, of your choosing.

I will be working with the Motor Vehicle Collisions - Crashes dataset from NYC OpenData (https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95/data). I have been noticing a lot more oversized pick up truck on the road and have also read that these large, high-clearance vehicles are responsible for an uptick in pedestrian related accidents and fatalities (https://www.outsideonline.com/2411345/suvs-trucks-deadly-cyclist-crashes).

 I will use this data set to examine if vehicle types within a dense city environment are related to pedestrian injuries and death due to accidents. I will examine possible confounding factors that could contribute to a deadly crash such as zip code, time of night, day of the week, time of day, and time of year (as an aproximation of road conditions). I will be using the variables number_of_pedestrians_killed and number of pedestrians_injured as independent variables and the remaining variables as explanatory variables, with a pirmary focus on vehicle type as a leading explanatory variable.

______________
# Data Exploration
Import your dataset into your .ipynb, create dataframes, and explore your data.  

Include: 

* Summary statistics means, medians, quartiles, 
* Missing value information
* Any other relevant information about the dataset.  



In [364]:
import pandas as pd
import urllib
import requests
import json
import numpy as np

#import the json dataset and save it in a pandas dataframe

u = "https://data.cityofnewyork.us/resource/h9gi-nx95.json"
with urllib.request.urlopen(u) as url:
    mvj = json.loads(url.read().decode())
    mvdf = pd.DataFrame(mvj)


Change the datatype of some columns so they are compatible with funcitons used in data exploration.

In [365]:

mvdf['crash_date']=mvdf['crash_date'].astype('datetime64')
mvdf['crash_time']=mvdf['crash_time'].astype('datetime64')
mvdf['latitude']=mvdf['latitude'].astype('float')
mvdf['longitude']=mvdf['longitude'].astype('float')
mvdf['number_of_persons_injured']=mvdf['number_of_persons_injured'].astype('int')
mvdf['number_of_persons_killed']=mvdf['number_of_persons_killed'].astype('int')
mvdf['number_of_pedestrians_injured']=mvdf['number_of_pedestrians_injured'].astype('int')
mvdf['number_of_pedestrians_killed']=mvdf['number_of_pedestrians_killed'].astype('int')
mvdf['number_of_cyclist_injured']=mvdf['number_of_cyclist_injured'].astype('int')
mvdf['number_of_cyclist_killed']=mvdf['number_of_cyclist_killed'].astype('int')
mvdf['number_of_motorist_injured']=mvdf['number_of_motorist_injured'].astype('int')
mvdf['number_of_motorist_killed']=mvdf['number_of_motorist_killed'].astype('int')




Get the timeframe of the dataset. The dataset ranges from December 16, 2017 through October 18, 2020.


In [366]:
mindate=mvdf['crash_date'].min()
maxdate = mvdf['crash_date'].max()

print('This data set ranges from')
print(mindate)
print(maxdate)

This data set ranges from
2018-10-02 00:00:00
2020-10-18 00:00:00


#Average number of pedestrians killed or injured per 100 crashes. 

We can see that there are 6.1 Pedestrians injured per 100 crashes and 0 pedestrian deathes per 100 crashes.

This tells us it may be a good idea to drop the number of deaths as an indpeendent veriable due to lower frequency 

We will then lean on injuries as a good barameter for impact on pedestrians.

In [367]:
pi=round(mvdf['number_of_pedestrians_injured'].mean()*100,4)
pk = round(mvdf['number_of_pedestrians_killed'].mean()*100,4)



print("The Average number of pedestrians injured per 100 crashes:")
print(pi)

print("The Average number of pedestrians killed per 100 crashes:")
print(pk)


The Average number of pedestrians injured per 100 crashes:
6.0
The Average number of pedestrians killed per 100 crashes:
0.0


Again we see that there are very few instances of pedestrian deaths from crashes based on the quartiles being listed at 0 below. The data will have to be altered in the next section to offer a dataset that is more worthy of analysis.

In [368]:

print(mvdf['number_of_pedestrians_injured'].describe())

count    1000.000000
mean        0.060000
std         0.241782
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         2.000000
Name: number_of_pedestrians_injured, dtype: float64


# Data Wrangling
Create a subset of your original data and perform the following.  

1. Modify multiple column names.

2. Look at the structure of your data – are any variables improperly coded? Such as strings or characters? Convert to correct structure if needed.

3. Fix missing and invalid values in data.

4. Create new columns based on existing columns or calculations.

5. Drop column(s) from your dataset.

6. Drop a row(s) from your dataset.

7. Sort your data based on multiple variables. 

8. Filter your data based on some condition. 

9. Convert all the string values to upper or lower cases in one column.

10. Check whether numeric values are present in a given column of your dataframe.

11. Group your dataset by one column, and get the mean, min, and max values by group. 
  * Groupby()
  * agg() or .apply()

12. Group your dataset by two columns and then sort the aggregated results within the groups. 

**You are free (and should) to add on to these questions.  Please clearly indicate in your assignment your answers to these questions.**

#Change column names

Some of the columns in the original datasets come with long names that are somewhat cumbersome to retype each time they are needed. Here I will change the names of our indpendent variables to shorter, more mangeable names.

In [369]:
mvdf.rename(columns={"number_of_pedestrians_injured": "ped_inj", "number_of_pedestrians_killed": "ped_kill"},inplace=True)

#Fill in blank values

I have already changed the datatypes of numerous fields in the data exploration section so I will move on to fixing invalid data. As a whole this data set does not have much invalid data fields. I notice the zip code an borough fields are often blank when the accident happens on a thruway or bridge. I will modify the blank zip codes to read '00000' to keep with the data sctructure of other zip codes and I will modify the Boroughs that are blank to say "No Borough - Major Road".

In [370]:
mvdf["zip_code"].fillna("00000", inplace = True)
mvdf["borough"].fillna("No Borough - Major Road", inplace = True)
mvdf['vehicle_type_code1'].unique

<bound method Series.unique of 0      Station Wagon/Sport Utility Vehicle
1      Station Wagon/Sport Utility Vehicle
2                                      Bus
3                                    Sedan
4                                    Sedan
                      ...                 
995    Station Wagon/Sport Utility Vehicle
996                                  Sedan
997                          Pick-up Truck
998                                  Sedan
999    Station Wagon/Sport Utility Vehicle
Name: vehicle_type_code1, Length: 1000, dtype: object>

#One Hot Encoding and Dropping Rows and Columns

In this section I will decipher the vehicle_type_1 field to determine what type of vehicle was involved. I will use pandas get_dummies to create new columns using one hot encoding in order to create dummy variables that will have a 1 if the field contians a substring of a verhicle type and a 0 if it doesn't.

Since I am interested in the rate that large civilian vehicles and their sales are impacting public safety I will only be looking at civilian owned cars and will be limiting the dataset to the following list (bike, motorcycle, sedan, convertible, Station Wagon, Sports Utility Vehicle, Pick-Up Truck or Van).

Then based on these new columns I will create a vehicle size category (small, medium, large) using the numpy select function. 

Finally I will remove the contriburing factor vehcile columns and the vehcil type code columns beyond one to clean up the dataset and make it easier to read.

In [371]:
# drop rows for non civlian vehicles
mvdf= mvdf[mvdf['vehicle_type_code1'].isin(['Bike', 'Morotcycle','Convertible', 'Sedan','Station Wagon/Sport Utility Vehicle','Pick-Up Truck', 'Van'])]

#create vehicle size category

mvdf['size']='NA'
conditions = [ 
              (mvdf['vehicle_type_code1'].isin(['Bike', 'Morotcycle'])),
              (mvdf['vehicle_type_code1'].isin(['Convertible', 'Sedan','Station Wagon/Sport Utility Vehicle'])),
              (mvdf['vehicle_type_code1'].isin(['Pick-Up Truck', 'Van']))]

values = ['small','medium','large']

mvdf['size']= np.select(conditions,values)

#create dummy variables for remining vehicle types through one hot encoding

d=pd.get_dummies(mvdf['vehicle_type_code1'])


#remove unecessary columns

drops =['contributing_factor_vehicle_1', 'contributing_factor_vehicle_2',
       'vehicle_type_code2','contributing_factor_vehicle_3',
       'vehicle_type_code_3', 'contributing_factor_vehicle_4',
       'vehicle_type_code_4', 'contributing_factor_vehicle_5']

for d in drops:
  mvdf = mvdf.drop(d, axis = 'columns')


#Sort data by Date ascending and number of pedestrians injured descending

In [372]:

mvdf = mvdf.sort_values(["crash_date", "ped_inj"], ascending = (True, False))



#Filter Data and convert to lower and upper case

Filter data based on if any pedestrians were injured and if the crash happend in Brooklyn

In [373]:
mvdf.loc[(mvdf['ped_inj']>0) & (mvdf['borough']=='brooklyn')]

# We can see the result above returns no results. If we investigate we see its because the borough column is in all caps
# We could search with all caps but to be safe we can alter the search condition to ensure we will capture the filter correctly

mvdf.loc[(mvdf['ped_inj']>0) & (mvdf['borough'].str.lower()=='brooklyn')]


#If we want our vehicle field to also be all upper case like borough we can user the upper()

mvdf['vehicle_type_code1']=mvdf['vehicle_type_code1'].str.upper()
mvdf.head()

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,number_of_persons_injured,number_of_persons_killed,ped_inj,ped_kill,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,collision_id,vehicle_type_code1,cross_street_name,vehicle_type_code_5,size
691,2018-10-02,2020-10-22 10:45:00,QUEENS,11355,40.747948,-73.815710,"{'latitude': '40.747948', 'longitude': '-73.81...",NaN,NaN,0,0,0,0,0,0,0,0,3992167,SEDAN,140-09 OAK AVENUE,NaN,medium
106,2018-10-07,2020-10-22 16:00:00,MANHATTAN,10019,NaN,NaN,NaN,west 59 street,columbus circle,0,0,0,0,0,0,0,0,3995172,SEDAN,NaN,NaN,medium
47,2018-11-14,2020-10-22 10:00:00,QUEENS,11356,40.781670,-73.829216,"{'latitude': '40.78167', 'longitude': '-73.829...",NaN,NaN,0,0,0,0,0,0,0,0,4020665,SEDAN,139-15 20 AVENUE,NaN,medium
318,2018-11-14,2020-10-22 10:01:00,No Borough - Major Road,00000,NaN,NaN,NaN,VERRAZANO BRIDGE UPPER,NaN,1,0,0,0,0,0,1,0,4021296,SEDAN,NaN,NaN,medium
83,2018-11-15,2020-10-22 18:00:00,BROOKLYN,11205,40.693226,-73.967850,"{'latitude': '40.693226', 'longitude': '-73.96...",WAVERLY AVENUE,MYRTLE AVENUE,1,0,1,0,0,0,0,0,4021604,STATION WAGON/SPORT UTILITY VEHICLE,NaN,NaN,medium


# Check for Numerics

Check to see if the borough fields has any numeric values in it to ensure it is populating correctly.

ref: https://stackoverflow.com/questions/54426845/how-to-check-if-a-pandas-dataframe-contains-only-numeric-column-wise/54427157

In [374]:
pd.to_numeric(mvdf['borough'], errors='coerce').notnull().all()

False

#Group by one Column

Here is where we will begin to analyze whether vehicle size has an impact on pedestrian injuries. We will take the average numner of pedestrian injuries based on the vehicle size category.

In [377]:
mvdf.groupby(['size'])['ped_inj'].mean()

#
#(mvdf.groupby(['cluster', 'org'], as_index=False).mean()
 #           .groupby('cluster')['time'].mean())

size
large     0.000000
medium    0.050725
small     0.111111
Name: ped_inj, dtype: float64

We can see by the results above that the opposite of our hyposthesis is true and smaller vehicles tend to lead to a higher percentage of pedestrian deaths than larger vehicles. 

Lets dive deeper and see if vehicle type has a larger role than the arbitrary vehicle size categories established above.




In [378]:
mvdf.groupby(['vehicle_type_code1'])['ped_inj'].mean()

vehicle_type_code1
BIKE                                   0.111111
CONVERTIBLE                            0.000000
SEDAN                                  0.046122
STATION WAGON/SPORT UTILITY VEHICLE    0.057307
VAN                                    0.000000
Name: ped_inj, dtype: float64

# Group by Two Columns

Caontrary to our hyposthesis it appears the that larger vehicles aren't associated with more injuries in pedestrians. Lets look and see if any borough is associated with pedestrian injuries as wellas vehicle. WE sort by the borough in albhebetical order and then the vehicle in reverse alphebetical order.

In [388]:
g=pd.DataFrame(mvdf.groupby(['borough','vehicle_type_code1'])['ped_inj'].mean().sort_index(ascending=True))

g.sort_values(["borough", "vehicle_type_code1"], ascending = (True, False))

ped_inj
borough                 vehicle_type_code1                           
BRONX                   VAN                                  0.000000
                        STATION WAGON/SPORT UTILITY VEHICLE  0.103448
                        SEDAN                                0.075472
                        BIKE                                 0.000000
BROOKLYN                VAN                                  0.000000
                        STATION WAGON/SPORT UTILITY VEHICLE  0.048387
                        SEDAN                                0.040816
                        BIKE                                 0.333333
MANHATTAN               VAN                                  0.000000
                        STATION WAGON/SPORT UTILITY VEHICLE  0.105263
                        SEDAN                                0.040816
                        CONVERTIBLE                          0.000000
                        BIKE                                 0.000000
No Borough - Major Road VAN                                  0.000000
                        STATION WAGON/SPORT UTILITY VEHICLE  0.029630
                        SEDAN                                0.049180
                        CONVERTIBLE                          0.000000
                        BIKE                                 0.000000
QUEENS                  STATION WAGON/SPORT UTILITY VEHICLE  0.082192
                        SEDAN                                0.037037
                        BIKE                                 0.000000
STATEN ISLAND           STATION WAGON/SPORT UTILITY VEHICLE  0.000000
                        SEDAN                                0.000000

## Data Visualization
Use pandas to create a data visualization of your choosing on your dataset.  You may use Pandas or other libraries to complete your visualization (plotly, plotly express, seaborn, etc.)

You must include, at the minimum three different plots





# Conclusions 
Provide some conclusions or insight gained from looking at your data.
